In [10]:
import pandas as pd
import MetaTrader5 as mt5
import talib as ta
from datetime import datetime,timedelta
from out_in_bb_backtest import setup_tsl,setup_stop_tp,setup_estr_2
import numpy as np
from backtesting import Strategy, Backtest
import warnings
warnings.filterwarnings("ignore")

if not mt5.initialize(path = r'C:\Program Files\Zero Markets MetaTrader 5 Terminal\terminal64.exe'):
    print("initialize() failed")
    mt5.shutdown()

ativo = 'EURUSD'
#ativo = 'WIN@N'
df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M5, 0, 40000))
df['Data'] = pd.to_datetime(df['time'], unit='s').apply(lambda x: str(x))
df = df[['Data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df_puro = df.rename(columns = {'real_volume':'vol'})
df2 = df_puro.copy()

In [11]:
df2

,Data,Open,High,Low,Close,Vol
0,2022-07-13 00:20:00,1.00324,1.00324,1.00305,1.00312,0
1,2022-07-13 00:25:00,1.00312,1.00333,1.00312,1.00330,0
2,2022-07-13 00:30:00,1.00333,1.00334,1.00330,1.00334,0
3,2022-07-13 00:35:00,1.00334,1.00340,1.00334,1.00338,0
4,2022-07-13 00:40:00,1.00338,1.00338,1.00324,1.00337,0
...,...,...,...,...,...,...
39995,2023-01-26 03:35:00,1.09085,1.09127,1.09085,1.09124,0
39996,2023-01-26 03:40:00,1.09125,1.09155,1.09125,1.09153,0
39997,2023-01-26 03:45:00,1.09153,1.09204,1.09151,1.09185,0
39998,2023-01-26 03:50:00,1.09185,1.09190,1.09175,1.09189,0


In [12]:
df_new = df[-10000:]
time_period = 20
var_bb = 2
pontos = 0.0008
rate_stop = 1
rate_tp = 1
spread = 0.00005
timeframe = 5

df=df_puro.copy()
bt = setup_stop_tp(df_new,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=False)
#bt = setup_tsl(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=False)

stats = bt.run()
bt.plot()

df_trades = stats['_trades']
#df_trades = df_trades.drop_duplicates('EntryTime')
df_trades['Hora_h'] = df_trades['EntryTime'].apply(lambda x: x.hour)
df_trades['resultado_binario'] = df_trades['ReturnPct'].apply(lambda x: 1 if x > 0 else 0)

print(f'Time_period: {time_period} -- var_bb: {var_bb} -- pontos: {pontos}--Rate_tp: {rate_tp}')
df_new_hora = df_trades[['Hora_h','resultado_binario']]
df_new_hora = df_trades.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

#df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

Time_period: 20 -- var_bb: 2 -- pontos: 0.0008--Rate_tp: 1
    Hora_h  qtd_operacoes  taxa_acerto
0        0              3     0.666667
1        2              2     0.000000
2        3              1     1.000000
3        4              1     1.000000
4        5              1     0.000000
5        6              2     0.000000
6        7              1     0.000000
7        8              1     0.000000
8        9              1     1.000000
9       10              2     1.000000
10      11              2     1.000000
11      12              1     0.000000
12      13              3     0.000000
13      14              3     0.333333
14      15              4     0.750000
15      16              4     0.750000
16      17              2     0.000000
17      19              1     0.000000
18      20              2     0.000000
19      21              3     0.333333
20      22              1     0.000000
21      23              2     1.000000


In [6]:
df_new_hora['qtd_operacoes'].sum()

152

In [4]:
df_trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Hora_h,resultado_binario
0,100632981,21,25,0.99371,0.99433,62392.44822,0.000624,2022-08-29 02:40:00,2022-08-29 03:00:00,0 days 00:20:00,2,1
1,100823610,66,79,0.99245,0.99185,-60494.16600,-0.000605,2022-08-29 06:25:00,2022-08-29 07:30:00,0 days 01:05:00,6,0
2,100683525,125,126,0.99323,0.99263,-60410.11500,-0.000604,2022-08-29 11:20:00,2022-08-29 11:25:00,0 days 00:05:00,11,0
3,99952482,247,252,0.99989,0.99927,-61970.53884,-0.000620,2022-08-29 21:30:00,2022-08-29 21:55:00,0 days 00:25:00,21,0
4,99942481,395,395,0.99937,0.99996,58966.06379,0.000590,2022-08-30 09:50:00,2022-08-30 09:50:00,0 days 00:00:00,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...
457,94937148,29749,29754,1.08314,1.08371,54114.17436,0.000526,2023-01-20 03:00:00,2023-01-20 03:25:00,0 days 00:25:00,3,1
458,95007283,29788,29808,1.08291,1.08350,56054.29697,0.000545,2023-01-20 06:15:00,2023-01-20 07:55:00,0 days 01:40:00,6,1
459,94957329,29853,29855,1.08407,1.08346,-57923.97069,-0.000563,2023-01-20 11:40:00,2023-01-20 11:50:00,0 days 00:10:00,11,0
460,95061738,29883,29884,1.08227,1.08166,-57987.66018,-0.000564,2023-01-20 14:10:00,2023-01-20 14:15:00,0 days 00:05:00,14,0


In [2]:
result = pd.read_excel('eurusd_1M_candlepattern_1.xlsx')
result.sort_values(by = 'Taxa_acerto', ascending=False).iloc[0:30]

,Timeframe,pontos,Rate_tp,Time_period,var_bb,N_trades,Taxa_acerto,Pontos_total,Retorno
40,5,0.0003,1,30,2.2,24,79.166667,0.00300,0.408371
22,5,0.0003,1,20,2.2,27,77.777778,0.00315,0.414885
28,5,0.0004,1,20,2.2,27,74.074074,0.00385,0.479761
46,5,0.0004,1,30,2.2,24,70.833333,0.00280,0.387445
34,5,0.0006,1,20,2.2,27,70.370370,0.00525,0.613182
26,5,0.0004,1,20,2.0,30,70.000000,0.00330,0.438512
32,5,0.0006,1,20,2.0,30,70.000000,0.00570,0.661196
4,5,0.0003,1,14,2.2,23,69.565217,0.00155,0.265158
16,5,0.0006,1,14,2.2,23,69.565217,0.00425,0.518955
2,5,0.0003,1,14,2.0,31,67.741935,0.00175,0.299754


In [3]:
for index,result_atual in result.loc[[40,22,28,46,29,41,21,23]].iterrows():
    time_period = result_atual['Time_period']
    var_bb = result_atual['var_bb']
    pontos = result_atual['pontos']
    rate_stop = 1
    rate_tp = result_atual['Rate_tp']
    retorno = result_atual['Retorno']
    Pontos_total = result_atual['Pontos_total']
    spread = 0.00005
    timeframe = 5


    df=df_puro.copy()
    bt = setup_estr_2(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=False)
    #bt = setup_tsl(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=False)

    stats = bt.run()

    df_trades = stats['_trades']
    #df_trades = df_trades.drop_duplicates('EntryTime')
    df_trades['Hora_h'] = df_trades['EntryTime'].apply(lambda x: x.hour)
    df_trades['resultado_binario'] = df_trades['ReturnPct'].apply(lambda x: 1 if x > 0 else 0)

    print(f'Time_period: {time_period} -- var_bb: {var_bb} -- pontos: {pontos}--Rate_tp: {rate_tp}-- Retorno: {round(retorno,2)} -- Pontos_total: {Pontos_total}')
    df_new_hora = df_trades[['Hora_h','resultado_binario']]
    df_new_hora = df_trades.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

    #df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
    print(df_new_hora)


Time_period: 30.0 -- var_bb: 2.2 -- pontos: 0.0003--Rate_tp: 1.0-- Retorno: 0.41 -- Pontos_total: 0.002999999999999997
    Hora_h  qtd_operacoes  taxa_acerto
0        0              2     0.500000
1        1              7     0.571429
2        2              8     0.250000
3        3              5     1.000000
4        4              8     0.375000
5        5              4     0.750000
6        6              3     0.333333
7        7             11     0.727273
8        8              8     0.500000
9        9              7     0.428571
10      10              8     0.250000
11      11              9     0.777778
12      12              9     0.666667
13      13             11     0.272727
14      14             12     0.500000
15      15              9     0.666667
16      16             11     0.454545
17      17              9     0.666667
18      18              6     0.666667
19      19              5     0.400000
20      20             10     0.600000
21      21             

In [4]:
new[new['signal'] != 0][['signal']].tail(10)

,signal
2023-01-13 11:30:00,1
2023-01-13 13:10:00,1
2023-01-13 15:30:00,2
2023-01-13 16:55:00,2
2023-01-13 18:15:00,1
2023-01-13 22:55:00,2
2023-01-16 03:50:00,2
2023-01-16 06:50:00,1
2023-01-16 08:35:00,1
2023-01-16 13:55:00,1


In [ ]:
stats['_trades'].tail(30)

#### Setup_Tsl

In [2]:
result = pd.read_excel('resultado_win_1M_ema_False_tls.xlsx')
result.sort_values(by = 'Retorno', ascending=False).iloc[:]

,Timeframe,pontos,Time_period,var_bb,N_trades,Taxa_acerto,Retorno
0,5,250,14,1.8,917,35.659760,12.770385
7,5,250,30,2.0,576,34.895833,7.520610
6,5,250,30,1.8,614,36.319218,6.898800
3,5,250,20,1.8,764,34.162304,0.399000
8,5,250,30,2.2,503,33.797217,-0.200325
2,5,250,14,2.2,797,35.257215,-0.266115
1,5,250,14,2.0,906,34.105960,-4.984950
4,5,250,20,2.0,703,33.997155,-5.327970
5,5,250,20,2.2,656,34.146341,-8.381820


In [3]:
for index,result_atual in result.loc[[0,7,6]].iterrows():
    time_period = result_atual['Time_period']
    var_bb = result_atual['var_bb']
    pontos = result_atual['pontos']
    rate_stop = 1
    #rate_tp = result_atual['Rate_tp']
    retorno = result_atual['Retorno']
    #Pontos_total = result_atual['Pontos_total']
    spread = 0.00005
    timeframe = 5


    df=df_puro.copy()
    bt = setup_tsl(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,EMA=True)

    stats = bt.run()

    df_trades = stats['_trades']
    df_trades = df_trades.drop_duplicates('EntryTime')
    df_trades['Hora_h'] = df_trades['EntryTime'].apply(lambda x: x.hour)
    df_trades['resultado_binario'] = df_trades['ReturnPct'].apply(lambda x: 1 if x > 0 else 0)

    print(f'Time_period: {time_period} -- var_bb: {var_bb} -- pontos: {pontos}-- Retorno: {round(retorno,2)}')
    df_new_hora = df_trades[['Hora_h','resultado_binario']]
    df_new_hora = df_trades.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

    #df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
    print(df_new_hora)



Time_period: 14.0 -- var_bb: 1.8 -- pontos: 250.0-- Retorno: 12.77
   Hora_h  qtd_operacoes  taxa_acerto
0       9             58     0.362069
1      10             72     0.347222
2      11             61     0.360656
3      12             58     0.241379
4      13             62     0.435484
5      14             58     0.344828
6      15             51     0.372549
7      16             59     0.305085
8      17             54     0.333333
9      18              1     1.000000
Time_period: 30.0 -- var_bb: 2.0 -- pontos: 250.0-- Retorno: 7.52
   Hora_h  qtd_operacoes  taxa_acerto
0       9             19     0.157895
1      10             27     0.370370
2      11             21     0.380952
3      12             28     0.178571
4      13             21     0.476190
5      14             25     0.440000
6      15             20     0.350000
7      16             30     0.166667
8      17             19     0.421053
9      18              1     1.000000
Time_period: 30.0 -- var_bb: 1.

In [15]:
bt.plot()

Row(id='1418', ...)

In [ ]:
df_trades = stats['_trades']
df_trades['Hora_h'] = df_trades['EntryTime'].apply(lambda x: x.hour)
df_trades['resultado_binario'] = df_trades['ReturnPct'].apply(lambda x: 1 if x > 0 else 0)

print(f'Time_period: {time_period} -- var_bb: {var_bb} -- pontos: {pontos}-- Retorno: {round(retorno,2)} ')
df_new_hora = df_trades[['Hora_h','resultado_binario']]
df_new_hora = df_trades.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

#df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

Time_period: 20.0 -- var_bb: 2.0 -- pontos: 0.0006-- Retorno: 3.3 
    Hora_h  qtd_operacoes  taxa_acerto
0        0             44     0.522727
1        1             53     0.528302
2        2             40     0.275000
3        3             64     0.343750
4        4             36     0.361111
5        5             28     0.214286
6        6             47     0.361702
7        7             42     0.333333
8        8             57     0.368421
9        9             66     0.333333
10      10             56     0.375000
11      11             50     0.400000
12      12             46     0.347826
13      13             46     0.434783
14      14             52     0.403846
15      15             67     0.313433
16      16             49     0.387755
17      17             56     0.321429
18      18              3     0.666667
19      19             33     0.363636
20      20             51     0.352941
21      21             35     0.314286
22      22             30     0.2333